<a href="https://colab.research.google.com/github/snehal5797/Blockchain-IIOT/blob/master/Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

SA_Price_df = pd.read_csv('SA_Price.csv')
# SA_Price_df = SA_Price_df.rename(columns={"Dates": "date"})
Para_TMAX_df = pd.read_excel('Parafield_TMAX.xlsx')
# Para_TMAX_df = Para_TMAX_df.rename(columns={"Date": "date"})
# print(SA_Price_df.to_string())
# VAN_PAYOUT_TYPE="Vanila_Payout"
# CON_PAYOUT_TYPE="Vanila_Payout"

In [2]:
SA_Price_df. columns. values

array(['Dates', 'Price'], dtype=object)

In [3]:
Para_TMAX_df. columns. values

array(['Date', 'Tmax'], dtype=object)

In [4]:
#preprocessing of data

In [5]:
# from tqdm import tqdm
# tqdm.pandas()
# SA_Price_df['new_date'] = tqdm(SA_Price_df['Dates'].progress_apply(lambda x: pd.to_datetime(x).date()))

In [6]:
SA_Price_df['date'] = pd.to_datetime(SA_Price_df['Dates']).dt.date



In [7]:
SA_Price_df.head()

,Dates,Price,date
0,07-Dec-1998 01:40:00,0.0010,1998-12-07
1,07-Dec-1998 01:45:00,0.0010,1998-12-07
2,07-Dec-1998 01:50:00,16.8672,1998-12-07
3,07-Dec-1998 01:55:00,24.4500,1998-12-07
4,07-Dec-1998 02:00:00,25.3668,1998-12-07


In [8]:
SA_Price_df. columns. values

array(['Dates', 'Price', 'date'], dtype=object)

In [9]:
Para_TMAX_df['date'] = pd.to_datetime(Para_TMAX_df['Date']).dt.date


In [10]:
# Perform an outer join on the "date" column
merged_df = pd.merge(SA_Price_df, Para_TMAX_df, on='date', how='outer')

merged_df.head()

,Dates,Price,date,Date,Tmax
0,07-Dec-1998 01:40:00,0.0010,1998-12-07,1998-12-07,29.3
1,07-Dec-1998 01:45:00,0.0010,1998-12-07,1998-12-07,29.3
2,07-Dec-1998 01:50:00,16.8672,1998-12-07,1998-12-07,29.3
3,07-Dec-1998 01:55:00,24.4500,1998-12-07,1998-12-07,29.3
4,07-Dec-1998 02:00:00,25.3668,1998-12-07,1998-12-07,29.3


##new way around

In [11]:
merged_df['Dates'] = pd.to_datetime(merged_df['Dates'], errors='coerce')

In [12]:
#merged_df = merged_df.drop(columns=['new_date'])
#merged_df = merged_df.drop(columns=['Date'])
#merged_df = merged_df.drop(columns=['Dates'])

#Calculation :

In [13]:
MW_Notional=100
Power_Price_Strike=100
Temperature=41

In [14]:
from numpy import maximum

In [15]:
merged_df["Final_price"] = MW_Notional * 1/12 * maximum(0, merged_df["Price"]-Power_Price_Strike)
#merged_df['Dates'] = pd.to_datetime(merged_df['Dates'], errors='coerce')

Normal = merged_df[merged_df["Dates"].dt.quarter == 1]
Normal["year"] = Normal["Dates"].dt.year

# # Group the data by year and sum the payouts for each year
Normal = Normal.groupby("year").sum()["Final_price"]

<ipython-input-15-0eeeeb7fef7a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Normal["year"] = Normal["Dates"].dt.year


In [16]:
Normal.head()

year
1999    3.176967e+06
2000    6.392693e+06
2001    9.298490e+06
2002    9.140873e+04
2003    5.086377e+05
Name: Final_price, dtype: float64

##Calculate the Raw Payouts for Quanto

In [17]:
quanto = merged_df[merged_df["Tmax"] > Temperature ]
# quanto = quanto[quanto["Dates"].dt.quarter == 1]

# Add year column to the dataframe
quanto["year"] = quanto["Dates"].dt.year

# Group the data by year and sum the price for each year
yearly_sum = quanto.groupby("year").sum()["Final_price"]

print(yearly_sum)

year
1998.0    4.370545e+06
1999.0    5.708216e+05
2000.0    3.997208e+06
2001.0    5.713555e+06
2002.0    2.642797e+05
2003.0    3.333317e+05
2004.0    1.168486e+06
2005.0    8.294653e+05
2006.0    1.872926e+06
2007.0    5.484567e+05
2008.0    3.483695e+06
2009.0    1.576342e+07
2010.0    2.005521e+05
2011.0    5.527752e+06
2012.0    0.000000e+00
2013.0    2.403734e+06
2014.0    2.697570e+06
2015.0    5.285323e+05
2016.0    1.041000e+04
2017.0    3.377079e+06
2018.0    5.162264e+06
2019.0    1.325302e+07
2020.0    1.800499e+06
2021.0    2.452148e+05
2022.0    3.180499e+05
Name: Final_price, dtype: float64


<ipython-input-17-825004870829>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quanto["year"] = quanto["Dates"].dt.year


In [18]:
yearly_sum.head()

year
1998.0    4.370545e+06
1999.0    5.708216e+05
2000.0    3.997208e+06
2001.0    5.713555e+06
2002.0    2.642797e+05
Name: Final_price, dtype: float64

In [29]:
import numpy as np


In [31]:
print(np.array(yearly_sum.keys().tolist()))


[1998. 1999. 2000. 2001. 2002. 2003. 2004. 2005. 2006. 2007. 2008. 2009.
 2010. 2011. 2012. 2013. 2014. 2015. 2016. 2017. 2018. 2019. 2020. 2021.
 2022.]


In [33]:
print(np.array(yearly_sum.values.tolist()))

[4.37054497e+06 5.70821578e+05 3.99720848e+06 5.71355516e+06
 2.64279713e+05 3.33331703e+05 1.16848626e+06 8.29465256e+05
 1.87292569e+06 5.48456744e+05 3.48369466e+06 1.57634240e+07
 2.00552141e+05 5.52775215e+06 0.00000000e+00 2.40373396e+06
 2.69756993e+06 5.28532340e+05 1.04100000e+04 3.37707881e+06
 5.16226368e+06 1.32530160e+07 1.80049909e+06 2.45214791e+05
 3.18049865e+05]
